# python 2.5
- enum (finish from previous session)
- pytest
- typehints
- mypy
- let's download (python2.5.semaphore.py)[python2.5.semaphore.py]

# enum
Quickly review from last session:

In [6]:
from enum import Enum, auto

class Colors(Enum):
    red = 'red'
    orange = 'orange'
    green = 'green'
    orangegreen = 'orangegreen'

print(Colors)

<enum 'Colors'>


In [2]:
Colors.red == Colors.green

False

In [3]:
list(Colors)

[<Colors.red: 'red'>,
 <Colors.orange: 'orange'>,
 <Colors.green: 'green'>,
 <Colors.orangegreen: 'orangegreen'>]

In [4]:
Colors.red in Colors

True

##### exercise
create `FlowerColors` and try yourself: `FlowerColors.red == Colors.red`

In [52]:
class FlowerColors(Enum):
    red = 'red'

FlowerColors.red == Colors.red

False

### auto values in Enums

In [13]:
class AnotherColors(Enum):
    red = auto()
    orange = auto()
    green = auto()
    orangegreen = auto()

list(AnotherColors)  # vs. list(Colors)

[<AnotherColors.red: 1>,
 <AnotherColors.orange: 2>,
 <AnotherColors.green: 3>,
 <AnotherColors.orangegreen: 4>]

# typehints

In [14]:
# %load python2.4.mypy.py
from typing import List, Dict
# def indent(size, content: List[str]):

def indent(size, content):
    print(f'This is output of {len(content)} lines')
    for line in content:
        print(size * '.' + line)


indent(3, 'micka masa mirek')
indent(3, 'micka masa mirek'.split())

This is output of 16 lines
...m
...i
...c
...k
...a
... 
...m
...a
...s
...a
... 
...m
...i
...r
...e
...k
This is output of 3 lines
...micka
...masa
...mirek


In [17]:
def indent_dict(content: List[Dict[str, int]]):
    for item in content:
        for key, value in item.items():
            print(key)
            c = value / 2
            print(c)


dict_content = [
    {'age': 5},
    # vs {'age': '5'},  # notice str instead of int
]

indent_dict(dict_content)

age
2.5


# mypy - static analysis types

In [1]:
!pip install mypy

  Using cached mypy_extensions-0.4.3-py2.py3-none-any.whl (4.5 kB)
  Using cached typing_extensions-3.7.4.3-py3-none-any.whl (22 kB)


You should consider upgrading via the 'c:\prace\venv\scripts\python.exe -m pip install --upgrade pip' command.


In [8]:
!mypy python2.4.mypy.py

Success: no issues found in 1 source file


In [7]:
!mypy --strict python2.4.mypy.py

python2.4.mypy.py:4: error: Function is missing a type annotation
python2.4.mypy.py:10: error: Call to untyped function "indent" in typed context
python2.4.mypy.py:11: error: Call to untyped function "indent" in typed context
python2.4.mypy.py:14: error: Function is missing a return type annotation
Found 4 errors in 1 file (checked 1 source file)


# test driven development (TDD)
- write tests ahead of actual code
- https://en.wikipedia.org/wiki/Test-driven_development#Test-driven_development_cycle

# how to test - case study

Let's extend previous example from Street light to Traffic semaphore.

Requirements:
1. defined state after creation is `red`
1. `change_status()` to move to another state
1. `get_status()` to return current state
1. `set_status(Colors.*)` to set current state by passing color from enum `Colors`

In [1]:
# semaphore.py
from enum import Enum #, auto

class Colors(Enum):
    red = 'red'
    orange = 'orange'
    green = 'green'
    orangered = 'orangered'


class Semaphore:
    sequence = [
        Colors.red,
        Colors.orangered,
        Colors.green,
        Colors.orange
    ]
    
    def __init__(self):
        self.current_index = 0

    def change_state(self):
        self.current_index += 1
    
    def get_status(self):
        return self.sequence[self.current_index]
    
    def set_status(self, status):
        index = self.sequence.index(status)
        self.current_index = index

In [5]:
sem = Semaphore()

print(sem.get_status())

sem.change_state()
print(sem.get_status())

sem.set_status(Colors.green)
print(sem.get_status())

Colors.red
Colors.orangered
Colors.green


In [ ]:
status = sem.get_status()
print('Status is ' + status)

## Why we need test-runner?

Naive test suite:

In [14]:
sem = Semaphore()

status = sem.get_status()
if status == Colors.red:
    print('ok - init status is red')
else:
    print('fail - init statu si NOT red')


sem.change_state()
#status = sem.get_status()
status = sem.change_state()
if status == Colors.orangered:
    print('ok - next status is orangered')
else:
    print('fail - next status si NOT orangered')

ok - init status is red
fail - next status si NOT orangered


# testing

* condition no.0: tests has to exists
* test each public method isn't ideal, better to test class behaviour as whole
* writing tests are like experimenting in science, with difference you know correct result in advance.

recommendations with examples \
`>>>` [python2.5.testing.notes.ipynb](python2.5.testing.notes.ipynb)

# pytest
- `pytest` is **test runner**
- test case as function
- tests discovery
- junit export
- fixture
- better than builtin `unittests`

**Exercise:** figure out serveral tests (`def test_...(self): ...`)\
*type single `X` per each test case (as comment in Barevné lístečky)*

In [10]:
# 7 test cases so far...
def test_initial_state_is_red(): ...
def test_next_state_after_red(): ...
def test_next_state_after_orangered(): ...
def test_next_state_after_green(): ...
def test_next_state_after_orange(): ...
def test_full_cycle(): ...
def test_set_invalid_color(): ...

Notes:
* `initial_state_is_red` is great
* `next_state_after_red` is ok, better is to contain `_is_orangered`

* stdout is captured by default (`-s` to display anyway)
* show verbose output (`-v`) with percents
* `--lf` (`--last-failed`)
* `--setup-plan` (

In [ ]:
def test_initial_state_is_red():
    sem = Semaphore()
    assert sem.get_status() == Colors.red

* PyCharm integration
    * **Run pytest in \<filename\>** from right click in editor
    * execute single test case
    * auto re-run tests after change (💗), also with minimized panel
    * by default passed tests are hidden

### fixtures

In [16]:
from pytest import fixture


def semaphore():
    return Semaphore()


def red_semaphore(semaphore):
    semaphore.set_state(Colors.red)
    return semaphore


def test_red_sem(red_semaphore):
    assert red_semaphore == Colors.red


def test_change_from_red(red_semaphore):
    ...

### expecting exceptions

In [22]:
from pytest import raises

def test_exception():
    with raises(Exception):
        raise KeyError('this was not expected')
        #raise Exception('this was not expected')

# typehints
- `Map`: To annotate arguments it is preferred to use an abstract collection type such as Mapping.
- `List`: To annotate arguments it is preferred to use an abstract collection type such as Sequence or Iterable.
- note for generics https://docs.python.org/3/library/typing.html#generic-concrete-collections

In [ ]:
# %load python2.5.mypy.py
from typing import List, Dict
# def indent(size, content: List[str]):

def indent(size, content):
    print(f'This is output of {len(content)} lines')
    for line in content:
        print(size * '.' + line)


indent(3, 'micka masa mirek')
indent(3, 'micka masa mirek'.split())


def indent_dict(content: List[Dict[str, int]]):
    for item in content:
        for key, value in item.items():
            print(key)
            c = value / 2
            print(c)


dict_content = [
    # vs {'age': '5'},
    {'age': 5},
]

indent_dict(dict_content)

# mypy - static analysis types

In [1]:
!pip install mypy

  Using cached mypy_extensions-0.4.3-py2.py3-none-any.whl (4.5 kB)
  Using cached typing_extensions-3.7.4.3-py3-none-any.whl (22 kB)


You should consider upgrading via the 'c:\prace\venv\scripts\python.exe -m pip install --upgrade pip' command.


In [8]:
!mypy python2.5.mypy.py

Success: no issues found in 1 source file


In [7]:
!mypy --strict python2.5.mypy.py

python2.4.mypy.py:4: error: Function is missing a type annotation
python2.4.mypy.py:10: error: Call to untyped function "indent" in typed context
python2.4.mypy.py:11: error: Call to untyped function "indent" in typed context
python2.4.mypy.py:14: error: Function is missing a return type annotation
Found 4 errors in 1 file (checked 1 source file)


# own exception
- https://docs.python.org/3/library/exceptions.html#exception-hierarchy

In [20]:
class TrafficLightException(Exception):
    ...

    
class TrafficLight:
    ...
    
    def set_color(self, newcolor):
        if new_color not in Color:
            raise TrafficLightException

In [27]:
def decorate_print(msg):
    print('<🐍>', end='')
    print(msg, end='')
    print('</🐍>')

decorate_print('python code')

def describe_function(fn):
    print('Passed function looks like this:', fn)

describe_function(decorate_print)

<🐍>python code</🐍>
Passed function looks like this: <function decorate_print at 0x00000168CC5FB820>
